In [36]:
# !pip install dash_bootstrap_components
# !pip install dash_daq
# !pip install dash_mantine_components

# EJjemplos
# https://github.com/lomska/Visualizing-Global-Trade-Networks
# https://github.com/plotly/dash-sample-apps/tree/main/apps/dash-manufacture-spc-dashboard

In [37]:
#####################################################
# Part 1: Import needed packages
#####################################################
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq
import dash_mantine_components as dmc

In [38]:
#####################################################
# Part 2: Basic app information
#####################################################
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Stock Analysis Dashboard"
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",}

In [39]:
from helpers_db import run_sql, get_engine

engine = get_engine()
with engine.begin() as conn:
  balance = run_sql(conn, """SELECT	
                              sum(p.amount) AS investment,
                              sum(p.quote_last_quote * p.shares) AS valuation,
                              sum(p.quote_last_quote * p.shares) / sum(p.amount) - 1 AS profits_hist_perc,
                              sum(p.quote_last_quote * p.shares) - sum(p.amount) AS profit_hist_value,
                              sum(ticks_1d_ago.close * p.shares) / sum(p.amount) - 1 AS profits_1d_ago_perc,
                              sum(ticks_1d_ago.close * p.shares) - sum(p.amount) AS profit_1d_ago_value
                            FROM assets a
                              INNER JOIN positions p ON a.symbol = p.symbol
                              INNER JOIN LATERAL (
                                  SELECT close, bucket FROM ticks_1d t1d
                                    WHERE p.symbol = t1d.symbol AND t1d.bucket < CURRENT_DATE ORDER BY bucket DESC LIMIT 1
                                ) AS ticks_1d_ago ON true
                            WHERE a.category = 'stock' AND p.shares > 0""").mappings().first()
  positions = run_sql(conn, """SELECT
                                p.symbol,
                                p.amount AS amount,
                                p.quote_purchase AS quote_purchase,
                                p.quote_last_quote AS quote_last_quote, ticks_1d_ago.close AS ticks_yesterday,
                                ((p.quote_last_quote * p.shares) - p.amount) AS profit_historical,
                                p.quote_last_quote/p.quote_purchase - 1 AS profit_historical_perc,
                                p.shares * (ticks_1d_ago.close - p.quote_last_quote) AS profits_last_day,
                                p.quote_last_quote/ticks_1d_ago.close - 1 AS profits_last_day_perc,
                                p.amount/p_tot.total AS participation
                              FROM assets a
                                INNER JOIN positions p ON a.symbol = p.symbol
                                INNER JOIN LATERAL (
                                    SELECT close, bucket FROM ticks_1d t1d
                                      WHERE p.symbol = t1d.symbol AND t1d.bucket < CURRENT_DATE ORDER BY bucket DESC LIMIT 1
                                  ) AS ticks_1d_ago ON true
                                JOIN LATERAL (SELECT SUM(amount) AS total FROM positions) AS p_tot ON true
                              WHERE a.category = 'stock' AND p.shares > 0
                              ORDER BY (p.quote_last_quote/ticks_1d_ago.close - 1)"""
                      ).mappings().all()
df = pd.DataFrame.from_dict(positions)
df

,amount,participation,profit_historical,profit_historical_perc,profits_last_day,profits_last_day_perc,quote_last_quote,quote_purchase,symbol,ticks_yesterday
0,849.500000,0.022694,-464.500000,-0.546792,35.0000,-0.083333,3.850,8.495000,SURG,4.200
1,541.500000,0.014466,-156.000000,-0.288089,34.5000,-0.082143,25.700,36.100000,GCT,28.000
2,1032.500000,0.027583,46.500000,0.045036,69.0000,-0.060105,5.395,5.162500,CRDF,5.740
3,659.000000,0.017605,-165.000000,-0.250379,30.0000,-0.057252,9.880,13.180000,NKTX,10.480
4,2235.370000,0.059717,1470.230000,0.657712,110.2800,-0.028900,926.400,558.842500,NVDA,953.970
5,633.840000,0.016933,-51.360000,-0.081030,12.2400,-0.020581,72.810,79.230000,ON,74.340
6,1409.500000,0.037654,79.400000,0.056332,21.7500,-0.014398,496.300,469.833333,META,503.550
7,1413.677000,0.037766,-331.427000,-0.234443,10.5000,-0.009609,14.430,18.849027,AAOI,14.570
8,507.346400,0.013554,-190.046400,-0.374589,2.8500,-0.008902,63.460,101.469280,SEDG,64.030
9,1446.510000,0.038643,-137.290000,-0.094911,10.7800,-0.008167,119.020,131.500909,ALB,120.000


In [40]:
#####################################################
# Part 4: App layout
#####################################################
color_negative = '#FF5E5E'
color_positive = '#B0C5A4'

div_dropdown_symbol = html.Div(dcc.Dropdown(
    id='symbol-dropdown', value=df.symbol.to_list(), clearable=False, multi=True,
    options=[{'label': x, 'value': x} for x in sorted(df.symbol.unique())]
), className='six columns', style={"width": "50%"}, )

div_dropdown_field = html.Div(dcc.Dropdown(
    id='field-dropdown', value='symbol', clearable=False,
    options=[{'label': x, 'value': x} for x in [i for j, i in enumerate(df.columns) if j not in ['symbol']]]
), className='six columns', style={"width": "25%"}, )

div_led_gains_today = html.Div(daq.LEDDisplay(
                        id="led-gains-today",
                        value='{:.3f}'.format(balance["profits_hist_perc"] * 100),
                        color= color_positive if balance["profits_hist_perc"] > 0 else color_negative,
                        # backgroundColor="#1e2130",
                        size=50,
                    ), className='six columns', style={"width": "50%"}, )
div_led_gains_1d_ago = html.Div(daq.LEDDisplay(
                            id="led-gains-1dago",
                            value='{:.3f}'.format(balance["profits_1d_ago_perc"] * 100),
                            color= color_positive if balance["profits_1d_ago_perc"] > 0 else color_negative,
                            # backgroundColor="#1e2130",
                            size=50,
                        ), className='six columns', style={"width": "50%"}, )

app.layout = html.Div([
    html.Div([div_led_gains_today, div_led_gains_1d_ago], className='row'),
    html.Div([html.Div([div_dropdown_symbol, div_dropdown_field], className='row'), ], className='custom-dropdown'),
    html.Div(html.Div(id='table-container_1'), style={'marginBottom': '15px', 'marginTop': '0px'}),
  ], style=CONTENT_STYLE)


In [41]:
money = dash_table.FormatTemplate.money(2)
percentage = dash_table.FormatTemplate.percentage(2)
columns = [
    dict(id='symbol', name='Symbol'),
    dict(id='amount', name='Amount', type='numeric', format=money),
    
    dict(id='quote_purchase', name='Purchase ($)', type='numeric', format=money),
    dict(id='quote_last_quote', name='Last Quote ($)', type='numeric', format=money),
    dict(id='ticks_yesterday', name='Yesterday ($)', type='numeric', format=money),
    
    dict(id='profit_historical', name='Profit ($)', type='numeric', format=money),
    dict(id='profits_last_day', name='Last Day ($)', type='numeric', format=money),
    
    dict(id='profit_historical_perc', name='Profit (%)', type='numeric', format=percentage),
    dict(id='profits_last_day_perc', name='Last Day (%)', type='numeric', format=percentage),
    dict(id='participation', name='Weight', type='numeric', format=percentage)
]
cols_colors = ['profit_historical','profit_historical_perc','profits_last_day','profits_last_day_perc']

@app.callback(
    Output("table", "style_data_conditional"),
    Input("table", "derived_virtual_selected_row_ids"),
)
def style_selected_rows(sel_rows):
    if sel_rows is None:                                                                                                                                                                                                                      
        return dash.no_update
    val = [
        {"if": {"filter_query": "{{id}} ={}".format(i)}, "backgroundColor": "#404040",}
        for i in sel_rows
    ]   
    return val

@app.callback(
    [Output('table-container_1', 'children')],
    [Input(component_id='symbol-dropdown', component_property='value'),
     Input(component_id='field-dropdown', component_property='value')]
)
def display_value(symbol_chosen, field_chosen):
  return (dash_table.DataTable(columns=columns,
                               id='datatable-positions',
                            #    style_data_conditional=[
                            #      {'if': { 'row_index': 'even', 'filter': 'row_index >num(2)' }, 
                            #       'backgroundColor': '#EBEDEF' },
                            #     ],
                               style_data_conditional=(
                                    [
                                        { 'if': { 'row_index': 'even', 'filter': 'row_index > num(2)' }, 'backgroundColor': '#EBEDEF' },
                                    ] + 
                                    [
                                        { 'if': { 'filter_query': '{{{}}} > 0'.format(col, value), 'column_id': col }, 'color': 'green'
                                        } for (col, value) in df[cols_colors].quantile(0.1).iteritems()
                                    ] +
                                    [
                                        { 'if': { 'filter_query': '{{{}}} < 0'.format(col, value), 'column_id': col }, 'color': 'tomato'
                                        } for (col, value) in df[cols_colors].iteritems()
                                    ] 
                                    # + [
                                    #     {
                                    #         "if": {"state": "selected"},              # 'active' | 'selected'
                                    #         "backgroundColor": "rgba(0, 116, 217, 0.3)",
                                    #         "border": "1px solid blue",
                                    #     }
                                    # ]
                                    
                                    
                                ),
                               data=df.sort_values(by=[field_chosen]).to_dict('records'),
                               export_format="csv",
                               style_as_list_view=True,
                               fill_width=True,
                               sort_action='native',
                               sort_mode='single',
                               style_cell={'font-size': '12px'},
                               style_header={'backgroundColor': 'black', 'color': 'white', },
                            #    fixed_columns={'headers': True, 'data': 1},
                               fixed_rows={'headers': True, 'data': 0},
                               
                               ),)

In [42]:
#####################################################
# Part 6: Set up local server to show the dashboard
#####################################################
if __name__ == '__main__':
    app.run_server(debug=False, port=8008)
    #   app.run(debug=True)

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:51: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:53: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:51: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:53: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:51: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_2307/1134791654.py:53